# Current-Corrected Vessel Velocities from EMSA Program AIS Messages

Exploration of processing of SalishSeaCast V2021-11 u & v current fields to calculate 
current-corrected vessel velocities from EMSA program AIS messages.

The `conda` environment in which this notebook runs is described in the `environment.yaml` file 
in this directory

In [7]:
from pathlib import Path

import pandas
import xarray

Use the OPeNDAP capability of `xarray.open_dataset()` to load the u & v fields dataset metadata
from the SalishSeaCast ERDDAP server.

In [2]:
u_url = "https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DuGridFields1hV21-11"
v_url = "https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DvGridFields1hV21-11"

u_ds = xarray.open_dataset(u_url)
v_ds = xarray.open_dataset(v_url)

In [3]:
u_ds

<xarray.Dataset>
Dimensions:    (time: 149976, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time       (time) datetime64[ns] 2007-01-01T00:30:00 ... 2024-02-09T23:30:00
  * depth      (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
  * gridY      (gridY) int16 0 1 2 3 4 5 6 7 ... 890 891 892 893 894 895 896 897
  * gridX      (gridX) int16 0 1 2 3 4 5 6 7 ... 390 391 392 393 394 395 396 397
Data variables:
    uVelocity  (time, depth, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-09T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-09 17:19:46 GMT
    title:                     Green, Salish Sea, 3d u Grid Variable Fields, ...
    uuid:                      4e3c7843-7c43-495a-b588-ccb317660597

In [4]:
v_ds

<xarray.Dataset>
Dimensions:    (time: 149976, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time       (time) datetime64[ns] 2007-01-01T00:30:00 ... 2024-02-09T23:30:00
  * depth      (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
  * gridY      (gridY) int16 0 1 2 3 4 5 6 7 ... 890 891 892 893 894 895 896 897
  * gridX      (gridX) int16 0 1 2 3 4 5 6 7 ... 390 391 392 393 394 395 396 397
Data variables:
    vVelocity  (time, depth, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-09T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-09 17:19:48 GMT
    title:                     Green, Salish Sea, 3d v Grid Variable Fields, ...
    uuid:                      36c847c8-9a0d-4c2c-8c5e-b0815b030413

Load the relevant columns from the AIS Excel file.

In [10]:
ais_data_path = Path("AIS_data.xlsx")

ais_df = pandas.read_excel(
    ais_data_path,
    usecols=['created','identity_id','speed','course','lat','lon','name','heading'],
    dtype={'lat': float,'lon': float}
)

In [11]:
ais_df

,created,identity_id,speed,course,lat,lon,name,heading
0,2024-01-16 09:44:52,636021537,9.9,188,48.530888,-123.218700,LAKE ANNECY,172.0
1,2024-01-16 07:56:44,357359000,6.4,16,48.643042,-123.231585,FEDERAL ILLINOIS,342.0
2,2024-01-16 07:53:52,563128300,5.8,14,48.470163,-123.168853,KAYING,347.0
3,2024-01-16 07:53:55,636019760,8.7,20,48.488953,-123.165610,GSL CHRISTEN,341.0
4,2024-01-16 07:59:55,255806029,9.3,139,48.785058,-123.017447,MSC NITYA B,224.0
...,...,...,...,...,...,...,...,...
1787,2024-01-16 07:47:51,563128300,5.7,354,48.451770,-123.166107,KAYING,5.0
1788,2024-01-15 16:14:52,256402000,7.4,14,48.623423,-123.232083,SATURALO,345.0
1789,2024-01-15 16:14:45,538007088,6.8,14,48.476035,-123.164172,BILLY JIM,341.0
1790,2024-01-16 08:23:45,357359000,6.8,293,48.712672,-123.190693,FEDERAL ILLINOIS,68.0


The SalishSeaCast grid region of interest for the QENTOL, YEN W̱SÁNEĆ Marine Guardians is
x = [220, 297] and y = [283, 350].

In [5]:
x_slice = slice(220, 297+1)
y_slice = slice(283, 350+1)

An alternative would be to grab the min/max lon/lat values from the AIS messages Excel file and use
https://github.com/SalishSeaCast/grid/blob/main/grid_from_lat_lon_mask999.nc to look up the corresponding
model grid region corners.

Use the creation times of the AIS messages to calculate the time slice we want the u & v fields for.
I'm assuming that the AIS times are UTC (which is what the SalishSeaCast times are).

In [27]:
time_min = ais_df["created"].min()
time_max = ais_df["created"].max()

time_min, time_max


(Timestamp('2024-01-15 16:11:46'), Timestamp('2024-01-17 10:29:54'))

In [25]:
u_surface = (
    u_ds
    .sel(time=slice(time_min, time_max), gridY=y_slice, gridX=x_slice)
    .sel(depth=0, method="nearest")
    )

u_surface

<xarray.Dataset>
Dimensions:    (time: 42, gridY: 69, gridX: 79)
Coordinates:
  * time       (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
    depth      float32 0.5
  * gridY      (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX      (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
Data variables:
    uVelocity  (time, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-09T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-09 17:19:46 GMT
    title:                     Green, Salish Sea, 3d u Grid Variable Fields, ...
    uuid:                      4e3c7843-7c43-495a-b588-ccb317660597

In [28]:
v_surface = (
    u_ds
    .sel(time=slice(time_min, time_max), gridY=y_slice, gridX=x_slice)
    .sel(depth=0, method="nearest")
    )

v_surface

<xarray.Dataset>
Dimensions:    (time: 42, gridY: 69, gridX: 79)
Coordinates:
  * time       (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
    depth      float32 0.5
  * gridY      (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX      (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
Data variables:
    uVelocity  (time, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-09T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-09 17:19:46 GMT
    title:                     Green, Salish Sea, 3d u Grid Variable Fields, ...
    uuid:                      4e3c7843-7c43-495a-b588-ccb317660597

Checking the model times selected:

In [29]:
u_surface.time[0], u_surface.time[-1]

(<xarray.DataArray 'time' ()>
 array('2024-01-15T16:30:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time     datetime64[ns] 2024-01-15T16:30:00
     depth    float32 0.5
 Attributes:
     _CoordinateAxisType:    Time
     actual_range:           [1.1676114e+09 1.7075214e+09]
     axis:                   T
     comment:                time values are UTC at the centre of the interval...
     coverage_content_type:  modelResult
     ioos_category:          Time
     long_name:              Time axis
     standard_name:          time
     time_origin:            01-JAN-1970 00:00:00,
 <xarray.DataArray 'time' ()>
 array('2024-01-17T09:30:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time     datetime64[ns] 2024-01-17T09:30:00
     depth    float32 0.5
 Attributes:
     _CoordinateAxisType:    Time
     actual_range:           [1.1676114e+09 1.7075214e+09]
     axis:                   T
     comment:                time values are UTC at the centre of the interval.

There is probably room for refinement of the time slice limits choice.
In this case, the hour-averaged current components at 16:30 are a reasonable choice for the
`time_min` value of 16:11:46.
On the other hand, the 10:30 field values might be a better choice than 09:30 for the `time_max` value
of 10:29:54.
That refinement would have to be done by rounding the AIS time values because we can't use
`.sel(..., method="nearest")` when the selection is a slice.